# **Construction of MicroRNA Networks Based on Inferred Interactions for TCGA-BRCA**

This notebook produces the AT_MN family of data artifacts 

- TCGA: The Cancer Genome Atlas  
- BRCA: Breast Invasive Carcinoma

# Import Libraries and Configurations

In [1]:
import os
import sys
from collections import defaultdict
from itertools import combinations

import numpy as np
import pandas as pd

# Add project root to Python's path
sys.path.append(os.path.abspath(os.path.join('..')))

from config import (
    ASSOCIATION_FILTERING_PARAMETERS,
    BRCA_INTERIM_FILES_DIRS,
    BRCA_PROCESSED_FILES_DIRS,
    CYTOSCAPE_PROCESSED_FILES_DIRS,
    INTERACTION_INFERENCE_PARAMETERS,
    INTERACTION_FILTERING_PARAMETERS,
)

# Interaction Networks

## Functions

In [2]:
def flag_inferred_interactions(dir_base_name):
    """
    Flag inferred molecular interactions based on correlation and q-value thresholds.

    Parameters
    ----------
    dir_base_name : str
        A key used to locate specific subdirectories in global path dictionaries
        (`BRCA_INTERIM_FILES_DIRS` and `BRCA_PROCESSED_FILES_DIRS`) for reading
        and saving interaction data.

    Returns
    -------
    pandas.DataFrame
        A DataFrame of inferred interactions with an added column `is_interaction_of_interest`, 
        indicating whether each interaction meets the filtering criteria.
    """
    # Define the group interim and processed directories path
    interim_dir_path = BRCA_INTERIM_FILES_DIRS[dir_base_name]
    processed_dir_path = BRCA_PROCESSED_FILES_DIRS[dir_base_name]
    
    # Create a DataFrame for the inferred interactions
    file_name = INTERACTION_INFERENCE_PARAMETERS['file-name']
    df_inferred_interactions = pd.read_csv(os.path.join(interim_dir_path, file_name))
    
    # Flag the inferred interactions of interest, i.e. those with correlation and 
    # q-value values below the thresholds
    max_correlation = INTERACTION_FILTERING_PARAMETERS['max_correlation']
    max_qvalue = INTERACTION_FILTERING_PARAMETERS['max_qvalue']
    df_inferred_interactions['is_interaction_of_interest'] = np.where(
        ((df_inferred_interactions['correlation'] < max_correlation)
         & (df_inferred_interactions['qvalue'] < max_qvalue)), 1, 0
    )
    
    # Store the DataFrame of flagged inferred interactions
    df_inferred_interactions.to_csv(
        os.path.join(processed_dir_path, file_name), index=False
    )
    
    return df_inferred_interactions

In [3]:
def create_interaction_network(group):
    """
    Construct and save a microRNA-messenger RNA interaction network for a given group.

    Parameters
    ----------
    group : str
        The name of the group. Used to determine the subdirectory for reading/writing files.

    Returns
    -------
    dict of pandas.DataFrame
        A dictionary with two keys:
        - 'edges': DataFrame of filtered interactions.
        - 'nodes': DataFrame of unique nodes.
    """
    # Define the group processed directory path
    dir_base_name = (group.lower()).replace(' ', '-')
    processed_dir_path = CYTOSCAPE_PROCESSED_FILES_DIRS[dir_base_name]
    
    # Flag the inferred interactions of interest
    df_inferred_interactions = flag_inferred_interactions(dir_base_name)
    
    # Create the edges DataFrame
    df_edges = df_inferred_interactions \
        .query('is_interaction_of_interest == 1') \
        .drop(columns=['is_interaction_of_interest', 'accession_id', 'pvalue']) \
        .rename(columns={'mirna_name': 'source', 'gene_name': 'target'}) \
        .fillna(value={'mirtarbase': ''}) \
        .sort_values(by=['correlation', 'qvalue'], ascending=True) \
        .reset_index(drop=True)
    
    # Create the microRNA nodes DataFrame
    df_mir_nodes = df_edges[['source']].copy() \
        .rename(columns={'source': 'id'}) \
        .drop_duplicates(ignore_index=True)
    df_mir_nodes['label'] = df_mir_nodes['id']
    df_mir_nodes['type'] = 'MicroRNA'
    
    # Create the messenger RNA nodes DataFrame
    df_rna_nodes = df_edges[['target']].copy() \
        .rename(columns={'target': 'id'}) \
        .drop_duplicates(ignore_index=True)
    df_rna_nodes['label'] = df_rna_nodes['id']
    df_rna_nodes['type'] = 'Messenger RNA'
    
    # Create the nodes DataFrame
    df_nodes = pd.concat([df_mir_nodes, df_rna_nodes], ignore_index=True)
    
    # Store the DataFrames of edges
    edges_file_name = INTERACTION_FILTERING_PARAMETERS['edges_file_name']
    df_edges.to_csv(
        os.path.join(processed_dir_path, edges_file_name), index=False
    )
    
    # Store the DataFrames of nodes
    nodes_file_name = INTERACTION_FILTERING_PARAMETERS['nodes_file_name']
    df_nodes.to_csv(
        os.path.join(processed_dir_path, nodes_file_name), index=False
    )
    
    return {
        'edges': df_edges,
        'nodes': df_nodes,
    }

## Basal-like Files

In [4]:
# Create the files for the construction of the Basal-like tumor tissue network
files = create_interaction_network('Basal-like')

In [5]:
# Print the DataFrame of edges for the group network
files['edges']

,source,target,mirtarbase,correlation,qvalue
0,hsa-miR-182-5p,RAB27A,,-0.563990,0.000264
1,hsa-miR-18a-5p,ORAI3,MIRT050737,-0.544944,0.000498
2,hsa-miR-18a-5p,ZBTB4,,-0.532223,0.000771
3,hsa-miR-96-5p,RAB27A,,-0.495626,0.004446
4,hsa-miR-106b-5p,ATXN1,MIRT441140,-0.494514,0.004446
...,...,...,...,...,...
291,hsa-miR-125b-5p,DNAL4,,-0.356291,0.049799
292,hsa-miR-20a-5p,PRKACB,MIRT080178,-0.356091,0.049930
293,hsa-miR-148a-3p,DYNLL2,MIRT025997,-0.356036,0.049930
294,hsa-miR-106b-5p,TGFBR2,MIRT091689,-0.355909,0.049930


In [6]:
# Print the DataFrame of nodes for the group network
files['nodes']

,id,label,type
0,hsa-miR-182-5p,hsa-miR-182-5p,MicroRNA
1,hsa-miR-18a-5p,hsa-miR-18a-5p,MicroRNA
2,hsa-miR-96-5p,hsa-miR-96-5p,MicroRNA
3,hsa-miR-106b-5p,hsa-miR-106b-5p,MicroRNA
4,hsa-miR-221-3p,hsa-miR-221-3p,MicroRNA
...,...,...,...
286,B4GALT5,B4GALT5,Messenger RNA
287,DNAL4,DNAL4,Messenger RNA
288,DYNLL2,DYNLL2,Messenger RNA
289,TGFBR2,TGFBR2,Messenger RNA


## HER2-enriched Files

In [7]:
# Create the files for the construction of the HER2-enriched tumor tissue network
files = create_interaction_network('HER2-enriched')

In [8]:
# Print the DataFrame of edges for the group network
files['edges']

,source,target,mirtarbase,correlation,qvalue
0,hsa-miR-141-3p,IGF2,,-0.587491,0.032134
1,hsa-let-7d-5p,MEF2D,MIRT735654,-0.562406,0.032134
2,hsa-miR-378a-3p,ACP6,,-0.561791,0.032134
3,hsa-miR-18a-5p,ZBTB20,,-0.558715,0.032134
4,hsa-miR-106b-5p,PLXDC2,,-0.558237,0.032134
5,hsa-let-7d-5p,SMARCC1,,-0.546275,0.041507
6,hsa-miR-106b-5p,SALL1,,-0.544908,0.041507
7,hsa-let-7d-5p,HIF1AN,,-0.539303,0.041561
8,hsa-let-7d-5p,PRKAR2A,,-0.536637,0.041561
9,hsa-miR-155-5p,VAV3,,-0.536569,0.041561


In [9]:
# Print the DataFrame of nodes for the group network
files['nodes']

,id,label,type
0,hsa-miR-141-3p,hsa-miR-141-3p,MicroRNA
1,hsa-let-7d-5p,hsa-let-7d-5p,MicroRNA
2,hsa-miR-378a-3p,hsa-miR-378a-3p,MicroRNA
3,hsa-miR-18a-5p,hsa-miR-18a-5p,MicroRNA
4,hsa-miR-106b-5p,hsa-miR-106b-5p,MicroRNA
5,hsa-miR-155-5p,hsa-miR-155-5p,MicroRNA
6,hsa-miR-182-5p,hsa-miR-182-5p,MicroRNA
7,hsa-miR-320b,hsa-miR-320b,MicroRNA
8,hsa-miR-30c-5p,hsa-miR-30c-5p,MicroRNA
9,IGF2,IGF2,Messenger RNA


## Luminal A Files

In [10]:
# Create the files for the construction of the Luminal A tumor tissue network
files = create_interaction_network('Luminal A')

In [11]:
# Print the DataFrame of edges for the group network
files['edges']

,source,target,mirtarbase,correlation,qvalue
0,hsa-miR-200a-3p,WWTR1,,-0.477429,8.770978e-10
1,hsa-miR-141-3p,PHYHIPL,,-0.451734,1.338599e-08
2,hsa-miR-96-5p,SGMS2,,-0.437820,5.059186e-08
3,hsa-miR-29b-3p,PCSK5,,-0.425329,1.256904e-07
4,hsa-miR-342-3p,FRMD4A,,-0.424572,1.256904e-07
...,...,...,...,...,...
254,hsa-miR-106b-5p,CYP2U1,,-0.300433,3.990725e-04
255,hsa-miR-17-5p,SAR1B,,-0.300325,3.998514e-04
256,hsa-miR-200b-3p,SLC24A4,,-0.300305,3.998514e-04
257,hsa-miR-96-5p,SH3PXD2B,,-0.300204,4.013699e-04


In [12]:
# Print the DataFrame of nodes for the group network
files['nodes']

,id,label,type
0,hsa-miR-200a-3p,hsa-miR-200a-3p,MicroRNA
1,hsa-miR-141-3p,hsa-miR-141-3p,MicroRNA
2,hsa-miR-96-5p,hsa-miR-96-5p,MicroRNA
3,hsa-miR-29b-3p,hsa-miR-29b-3p,MicroRNA
4,hsa-miR-342-3p,hsa-miR-342-3p,MicroRNA
...,...,...,...
246,LUZP1,LUZP1,Messenger RNA
247,CYBRD1,CYBRD1,Messenger RNA
248,TBX15,TBX15,Messenger RNA
249,SLC24A4,SLC24A4,Messenger RNA


## Luminal B Files

In [13]:
# Create the files for the construction of the Luminal B tumor tissue network
files = create_interaction_network('Luminal B')

In [14]:
# Print the DataFrame of edges for the group network
files['edges']

,source,target,mirtarbase,correlation,qvalue
0,hsa-miR-200c-3p,ZMAT3,MIRT209182,-0.493395,0.000212
1,hsa-let-7g-5p,KLHL13,,-0.467255,0.000693
2,hsa-miR-30d-5p,FST,,-0.461213,0.000693
3,hsa-miR-200c-3p,FAM126A,,-0.458226,0.000693
4,hsa-miR-151a-3p,FRK,MIRT508167,-0.455802,0.000693
...,...,...,...,...,...
579,hsa-miR-339-5p,DAAM2,,-0.300442,0.030371
580,hsa-miR-30b-5p,TMTC3,,-0.300347,0.030439
581,hsa-miR-205-5p,RAB11FIP1,MIRT505979,-0.300243,0.030486
582,hsa-miR-222-3p,PANK3,MIRT046819,-0.300208,0.030486


In [15]:
# Print the DataFrame of nodes for the group network
files['nodes']

,id,label,type
0,hsa-miR-200c-3p,hsa-miR-200c-3p,MicroRNA
1,hsa-let-7g-5p,hsa-let-7g-5p,MicroRNA
2,hsa-miR-30d-5p,hsa-miR-30d-5p,MicroRNA
3,hsa-miR-151a-3p,hsa-miR-151a-3p,MicroRNA
4,hsa-miR-27b-3p,hsa-miR-27b-3p,MicroRNA
...,...,...,...
554,AFF2,AFF2,Messenger RNA
555,DAAM2,DAAM2,Messenger RNA
556,RAB11FIP1,RAB11FIP1,Messenger RNA
557,PANK3,PANK3,Messenger RNA


## Paired Normal Files

In [16]:
# Create the files for the construction of the Paired Normal tissue network
files = create_interaction_network('Paired Normal')

In [17]:
# Print the DataFrame of edges for the group network
files['edges']

,source,target,mirtarbase,correlation,qvalue
0,hsa-miR-22-3p,RPS3,,-0.815995,3.787009e-10
1,hsa-miR-452-5p,RPA1,,-0.809911,3.923583e-10
2,hsa-miR-141-3p,TRHDE,,-0.807245,3.923583e-10
3,hsa-miR-22-3p,RPL14,,-0.804375,4.216212e-10
4,hsa-miR-22-3p,LRRC1,MIRT030660,-0.800410,5.308921e-10
...,...,...,...,...,...
5067,hsa-miR-93-5p,CREBRF,MIRT028082,-0.332468,4.991622e-02
5068,hsa-miR-93-5p,USP37,,-0.332468,4.991622e-02
5069,hsa-miR-139-5p,ZC3H12A,,-0.332468,4.991622e-02
5070,hsa-miR-326,FAM168A,,-0.332468,4.991622e-02


In [18]:
# Print the DataFrame of nodes for the group network
files['nodes']

,id,label,type
0,hsa-miR-22-3p,hsa-miR-22-3p,MicroRNA
1,hsa-miR-452-5p,hsa-miR-452-5p,MicroRNA
2,hsa-miR-141-3p,hsa-miR-141-3p,MicroRNA
3,hsa-miR-200a-3p,hsa-miR-200a-3p,MicroRNA
4,hsa-miR-224-5p,hsa-miR-224-5p,MicroRNA
...,...,...,...
2910,NKAPD1,NKAPD1,Messenger RNA
2911,SLITRK4,SLITRK4,Messenger RNA
2912,PHF21A,PHF21A,Messenger RNA
2913,ZC3H12A,ZC3H12A,Messenger RNA


# Association Networks

## Functions

In [19]:
def infer_associations(dir_base_name):
    """
    Infer microRNA-microRNA associations based on shared interaction targets using the Jaccard index.

    Parameters
    ----------
    dir_base_name : str
        A key used to locate subdirectories in the global path dictionaries
        `CYTOSCAPE_PROCESSED_FILES_DIRS` and `BRCA_INTERIM_FILES_DIRS`.

    Returns
    -------
    pandas.DataFrame
        A DataFrame of inferred associations.
    """
    # Create a DataFrame for the interactions of interest
    dir_path = CYTOSCAPE_PROCESSED_FILES_DIRS[dir_base_name]
    file_path = os.path.join(
        dir_path, INTERACTION_FILTERING_PARAMETERS['edges_file_name']
    )
    df_interactions = pd.read_csv(file_path)

    # Determine the set of neighbors for each node
    neighbors = defaultdict(set)
    for _, row in df_interactions.iterrows():
        neighbors[row['source']].add(row['target'])
    
    # Create a list to store the results
    results = list()

    # Compute the Jaccard index between each combination of different nodes
    for node_a, node_b in combinations(neighbors, 2):
        # Get the sets of neighbors from both nodes
        set_a = neighbors[node_a]
        set_b = neighbors[node_b]
        
        # Compute the Jaccard index between both sets
        intersection = len(set_a & set_b)
        union = len(set_a | set_b)
        jaccard = intersection / union if union else 0
        
        # Insert the result for this combination in the list
        results.append((node_a, node_b, jaccard))
    
    # Create a DataFrame for the inferred associations
    columns = ['node_a', 'node_b', 'association']
    df_associations = pd.DataFrame(results, columns=columns) \
        .sort_values(by='association', ascending=False)
    
    # Store the DataFrame of inferred associations
    dir_path = BRCA_INTERIM_FILES_DIRS[dir_base_name]
    file_name = ASSOCIATION_FILTERING_PARAMETERS['inferred_associations']
    df_associations.to_csv(os.path.join(dir_path, file_name), index=False)
    
    return df_associations

In [20]:
def flag_inferred_associations(dir_base_name):
    """
    Flag inferred associations based on a minimum Jaccard index threshold.

    Parameters
    ----------
    dir_base_name : str
        A key used to identify the appropriate directory paths from global
        dictionaries (`BRCA_PROCESSED_FILES_DIRS` and `ASSOCIATION_FILTERING_PARAMETERS`).

    Returns
    -------
    pandas.DataFrame
        A DataFrame of inferred associations.
    """
    # Infer the associations
    df_associations = infer_associations(dir_base_name)
        
    # Flag the associations of interest, i.e. those with index above the threshold
    min_association = ASSOCIATION_FILTERING_PARAMETERS['min_index']
    df_associations['is_association_of_interest'] = np.where(
        df_associations['association'] > min_association, 1, 0
    )
    
    # Store the DataFrame of flagged inferred associations
    dir_path = BRCA_PROCESSED_FILES_DIRS[dir_base_name]
    file_name = ASSOCIATION_FILTERING_PARAMETERS['inferred_associations']
    df_associations.to_csv(os.path.join(dir_path, file_name), index=False)
    
    return df_associations

In [21]:
def create_association_network(group):
    """
    Construct and save a microRNA–microRNA association network for a given group.

    Parameters
    ----------
    group : str
        The name of the group.

    Returns
    -------
    dict of pandas.DataFrame
        A dictionary with two keys:
        - 'edges': DataFrame of filtered associations.
        - 'nodes': DataFrame of unique microRNA nodes.
    """
    # Define the group processed directory path
    dir_base_name = (group.lower()).replace(' ', '-')
    dir_path = CYTOSCAPE_PROCESSED_FILES_DIRS[dir_base_name]
    
    # Infer the associations and flag the associations of inteterest
    df_associations = flag_inferred_associations(dir_base_name)
    
    # Create the edges DataFrame
    df_edges = df_associations \
        .query('is_association_of_interest == 1') \
        .drop(columns=['is_association_of_interest']) \
        .rename(columns={'node_a': 'source', 'node_b': 'target'}) \
        .round(2) \
        .reset_index(drop=True)
    
    # Create the nodes DataFrame
    df_source_nodes = df_edges[['source']].copy() \
        .rename(columns={'source': 'id'})
    df_target_nodes = df_edges[['target']].copy() \
        .rename(columns={'target': 'id'})
        
    df_nodes = pd.concat([df_source_nodes, df_target_nodes], ignore_index=True) \
        .drop_duplicates(ignore_index=True)
    df_nodes['label'] = df_nodes['id']
    df_nodes['type'] = 'MicroRNA'
    
    # Store the DataFrames of edges
    edges_file_name = ASSOCIATION_FILTERING_PARAMETERS['edges_file_name']
    df_edges.to_csv(
        os.path.join(dir_path, edges_file_name), index=False
    )
    
    # Store the DataFrames of nodes
    nodes_file_name = ASSOCIATION_FILTERING_PARAMETERS['nodes_file_name']
    df_nodes.to_csv(
        os.path.join(dir_path, nodes_file_name), index=False
    )
    
    return {
        'edges': df_edges,
        'nodes': df_nodes,
    }

## Basal-like Files

In [22]:
# Compute the associations that will be in the Basal-like tumor tissue network
files = create_association_network('Basal-like')

In [23]:
# Print the DataFrame of edges for the group network
files['edges']

,source,target,association
0,hsa-miR-19a-3p,hsa-miR-19b-3p,0.67
1,hsa-miR-17-5p,hsa-miR-20a-5p,0.57
2,hsa-miR-23a-3p,hsa-miR-151a-3p,0.50
3,hsa-miR-27a-3p,hsa-miR-128-3p,0.25
4,hsa-miR-221-3p,hsa-miR-222-3p,0.25
5,hsa-miR-20a-5p,hsa-miR-130b-3p,0.17
6,hsa-miR-29b-3p,hsa-miR-29a-3p,0.17
7,hsa-miR-200b-3p,hsa-miR-200c-3p,0.16
8,hsa-miR-18a-5p,hsa-miR-19b-3p,0.14
9,hsa-miR-106a-5p,hsa-miR-17-5p,0.14


In [24]:
# Print the DataFrame of nodes for the group network
files['nodes']

,id,label,type
0,hsa-miR-19a-3p,hsa-miR-19a-3p,MicroRNA
1,hsa-miR-17-5p,hsa-miR-17-5p,MicroRNA
2,hsa-miR-23a-3p,hsa-miR-23a-3p,MicroRNA
3,hsa-miR-27a-3p,hsa-miR-27a-3p,MicroRNA
4,hsa-miR-221-3p,hsa-miR-221-3p,MicroRNA
5,hsa-miR-20a-5p,hsa-miR-20a-5p,MicroRNA
6,hsa-miR-29b-3p,hsa-miR-29b-3p,MicroRNA
7,hsa-miR-200b-3p,hsa-miR-200b-3p,MicroRNA
8,hsa-miR-18a-5p,hsa-miR-18a-5p,MicroRNA
9,hsa-miR-106a-5p,hsa-miR-106a-5p,MicroRNA


## HER2-enriched Files

In [25]:
# Compute the associations that will be in the HER2-enriched tumor tissue network
files = create_association_network('HER2-enriched')

In [26]:
# Print the DataFrame of edges for the group network
files['edges']

,source,target,association


In [27]:
# Print the DataFrame of nodes for the group network
files['nodes']

,id,label,type


## Luminal A Files

In [28]:
# Compute the associations that will be in the Luminal A tumor tissue network
files = create_association_network('Luminal A')

In [29]:
# Print the DataFrame of edges for the group network
files['edges']

,source,target,association
0,hsa-miR-365b-3p,hsa-miR-365a-3p,1.00
1,hsa-miR-30c-5p,hsa-miR-324-5p,0.50
2,hsa-miR-20a-5p,hsa-miR-17-5p,0.27
3,hsa-miR-20a-5p,hsa-miR-92a-3p,0.25
4,hsa-miR-33b-5p,hsa-miR-33a-5p,0.20
5,hsa-miR-200a-3p,hsa-miR-141-3p,0.20
6,hsa-miR-193b-3p,hsa-miR-148b-3p,0.17
7,hsa-miR-33b-5p,hsa-miR-148b-3p,0.12
8,hsa-miR-29b-3p,hsa-miR-29c-3p,0.11
9,hsa-miR-193b-3p,hsa-miR-33b-5p,0.11


In [30]:
# Print the DataFrame of nodes for the group network
files['nodes']

,id,label,type
0,hsa-miR-365b-3p,hsa-miR-365b-3p,MicroRNA
1,hsa-miR-30c-5p,hsa-miR-30c-5p,MicroRNA
2,hsa-miR-20a-5p,hsa-miR-20a-5p,MicroRNA
3,hsa-miR-33b-5p,hsa-miR-33b-5p,MicroRNA
4,hsa-miR-200a-3p,hsa-miR-200a-3p,MicroRNA
5,hsa-miR-193b-3p,hsa-miR-193b-3p,MicroRNA
6,hsa-miR-29b-3p,hsa-miR-29b-3p,MicroRNA
7,hsa-miR-192-5p,hsa-miR-192-5p,MicroRNA
8,hsa-miR-365a-3p,hsa-miR-365a-3p,MicroRNA
9,hsa-miR-324-5p,hsa-miR-324-5p,MicroRNA


## Luminal B Files

In [31]:
# Compute the associations that will be in the Luminal B tumor tissue network
files = create_association_network('Luminal B')

In [32]:
# Print the DataFrame of edges for the group network
files['edges']

,source,target,association
0,hsa-miR-15a-5p,hsa-miR-30e-5p,1.00
1,hsa-miR-381-3p,hsa-miR-299-5p,1.00
2,hsa-miR-378a-3p,hsa-miR-378c,0.33
3,hsa-miR-151a-3p,hsa-miR-212-3p,0.33
4,hsa-miR-150-5p,hsa-miR-378c,0.33
5,hsa-miR-26a-5p,hsa-miR-26b-5p,0.33
6,hsa-miR-501-3p,hsa-miR-502-3p,0.29
7,hsa-let-7d-5p,hsa-miR-107,0.25
8,hsa-miR-1271-5p,hsa-miR-378c,0.25
9,hsa-miR-378a-3p,hsa-miR-150-5p,0.20


In [33]:
# Print the DataFrame of nodes for the group network
files['nodes']

,id,label,type
0,hsa-miR-15a-5p,hsa-miR-15a-5p,MicroRNA
1,hsa-miR-381-3p,hsa-miR-381-3p,MicroRNA
2,hsa-miR-378a-3p,hsa-miR-378a-3p,MicroRNA
3,hsa-miR-151a-3p,hsa-miR-151a-3p,MicroRNA
4,hsa-miR-150-5p,hsa-miR-150-5p,MicroRNA
5,hsa-miR-26a-5p,hsa-miR-26a-5p,MicroRNA
6,hsa-miR-501-3p,hsa-miR-501-3p,MicroRNA
7,hsa-let-7d-5p,hsa-let-7d-5p,MicroRNA
8,hsa-miR-1271-5p,hsa-miR-1271-5p,MicroRNA
9,hsa-let-7i-5p,hsa-let-7i-5p,MicroRNA


## Paired Normal Files

In [34]:
# Compute the associations that will be in the Paired Normal tissue network
files = create_association_network('Paired Normal')

In [35]:
# Print the DataFrame of edges for the group network
files['edges']

,source,target,association
0,hsa-miR-199b-3p,hsa-miR-199a-3p,1.00
1,hsa-miR-365a-3p,hsa-miR-365b-3p,1.00
2,hsa-miR-330-5p,hsa-miR-874-3p,0.50
3,hsa-miR-378a-3p,hsa-miR-378c,0.48
4,hsa-miR-141-3p,hsa-miR-200a-3p,0.44
5,hsa-let-7b-5p,hsa-let-7c-5p,0.38
6,hsa-miR-497-5p,hsa-miR-299-5p,0.33
7,hsa-miR-497-5p,hsa-miR-503-5p,0.33
8,hsa-miR-20b-5p,hsa-miR-106a-5p,0.32
9,hsa-miR-181b-5p,hsa-miR-181d-5p,0.31


In [36]:
# Print the DataFrame of nodes for the group network
files['nodes']

,id,label,type
0,hsa-miR-199b-3p,hsa-miR-199b-3p,MicroRNA
1,hsa-miR-365a-3p,hsa-miR-365a-3p,MicroRNA
2,hsa-miR-330-5p,hsa-miR-330-5p,MicroRNA
3,hsa-miR-378a-3p,hsa-miR-378a-3p,MicroRNA
4,hsa-miR-141-3p,hsa-miR-141-3p,MicroRNA
...,...,...,...
56,hsa-miR-17-5p,hsa-miR-17-5p,MicroRNA
57,hsa-miR-708-5p,hsa-miR-708-5p,MicroRNA
58,hsa-miR-125b-5p,hsa-miR-125b-5p,MicroRNA
59,hsa-miR-424-5p,hsa-miR-424-5p,MicroRNA
